In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.utils.extmath import randomized_svd
 
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from dotenv import dotenv_values 
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import scipy.io
from scipy.signal import cont2discrete
import pyvista as pv
import h5py
import mat73
import dask.array as da
from dask.distributed import Client, LocalCluster
import opinf
config = dotenv_values(".env")
from util import *


In [ ]:
####### Set up dask cluster #######
# Setup Dask client
# cluster = LocalCluster() # Launches a scheduler and workers locally
# client = Client(cluster) # Connect to distributed cluster and override default
client = Client('tcp://127.0.0.1:8786')
client.cluster

####### # Set up hyperparameters for all ROMs  #######
n_timesteps = 2000 # Number of timesteps to pull from each episode
dt = 0.001
n_train = 3 # Number of training episodes
n_test = 7 # Number of testing episodes
ns_ROM = np.arange(2,200,2) # Dimensions of ROM state space


In [ ]:
####### Set up filepaths #######
# Setup filepaths for reading data 
filepath = config["currentDirectory"] + "data/archivedDataSets/FullAssembly_Constrained_FullSetForRAL_goodMatParams/"
romDir = config["currentDirectory"] + "data/archivedDataSets/FullAssembly_Constrained_FullSetForRAL_goodMatParams/ROMs/"

In [ ]:
####### Load in data #######
print("Loading in data...")
# Read in all training data 
data = h5py.File(filepath+ "fullDataSet.hdf5", 'r')
X_fom = da.from_array(data["stateData"], chunks=(4096, 4096,1))
# X_fom = X_fom[:,:,0:n_train]
U_fom = da.from_array(data["inputData"], chunks=(4096, 4096,1))
# U_fom = U_fom[:,:,0:n_train]
Y_fom = da.from_array(data["reducedCenterlineData"], chunks=(4096, 4096,1))
# Y_fom = Y_fom[:,:,0:n_train]
x0 = data["stateData"][:,0,0]
y0 = data["reducedCenterlineData"][:,0,0]
n = X_fom.shape[0]
l = U_fom.shape[0]
m = Y_fom.shape[0]


In [ ]:
# Initialize arrays for storing RMS estimation errors 
num_ROMs = len(ns_ROM)
num_trials = n_test
rms_errors_era_ol = np.zeros((num_ROMs, num_trials))
rms_errors_dmdc_ol = np.zeros((num_ROMs, num_trials))

In [ ]:
# iterate over all ROM dimensions
for n_ROM in num_ROMs:
    # Load in ROM models
    era_file = romDir + "eraROM_" + str(n_ROM)+"_dim_" +str(n_train)+"train" +".npz"
    dmd_file = romDir + "dmdcSystemMatrices_" + str(n_ROM)+"_dim_" +str(n_train)+"train" + ".npz"
    eraData = np.load(era_file)
    dmdcData = np.load(dmd_file)
    A_era = eraData["A_era"]
    B_era = eraData["B_era"]
    C_era = eraData["C_era"]
    D_era = eraData["D_era"]
    x0_era = eraData["x0_era"]
    y0_era = eraData["y0_era"]
    A_dmdc = dmdcData["A_dmdc"]
    B_dmdc = dmdcData["B_dmdc"]
    C_dmdc = dmdcData["C_dmdc"]
    D_dmdc = dmdcData["D_dmdc"]
    x0_dmdc = dmdcData["x0_dmdc"]
    y0_dmdc = dmdcData["y0_dmdc"]
    basis_dmdc = dmdcData["basis_dmdc"]
    # Initialize array for holding output estimates over time
    Y_era_ol = np.zeros((m, n_timesteps, num_trials))
    Y_dmdc_ol = np.zeros((m, n_timesteps, num_trials))
    # Initialize array for holding state estimates over time
    X_era_ol = np.zeros((n_ROM, n_timesteps, num_trials))
    X_dmdc_ol = np.zeros((n_ROM, n_timesteps, num_trials))
    # Iterate over all testing episodes, solving for open loop estimates
    